# CSCI 4962 Project
Ethan Cruz

Presented on Monday, Dec 5 2022

In [1]:
# Imports
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Get data into panda dataframe and fix it up.

In [2]:
df = pd.read_csv('match_data_version1.csv')
df.head()

,Unnamed: 0,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,status.message,status.status_code
0,0,1.585155e+12,1323.0,4.247263e+09,CLASSIC,MATCHED_GAME,10.6.314.4405,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
1,1,1.585152e+12,1317.0,4.247156e+09,CLASSIC,MATCHED_GAME,10.6.314.4405,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
2,2,1.585059e+12,932.0,4.243963e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
3,3,1.584978e+12,2098.0,4.241678e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
4,4,1.584973e+12,2344.0,4.241539e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN


Remove ARAMs, 3v3s, etc. Keep only Classic matches and grab the participants row.

In [3]:
df = df[df['gameMode'] == 'CLASSIC']
df = df['participants']
df = df.dropna()
df.head()

0    [{'participantId': 1, 'teamId': 100, 'champion...
1    [{'participantId': 1, 'teamId': 100, 'champion...
2    [{'participantId': 1, 'teamId': 100, 'champion...
3    [{'participantId': 1, 'teamId': 100, 'champion...
4    [{'participantId': 1, 'teamId': 100, 'champion...
Name: participants, dtype: object

Turn into np array and print the first row

In [4]:
data = df.to_numpy()
print(data[0])

[{'participantId': 1, 'teamId': 100, 'championId': 7, 'spell1Id': 1, 'spell2Id': 4, 'stats': {'participantId': 1, 'win': False, 'item0': 2033, 'item1': 3285, 'item2': 3108, 'item3': 1082, 'item4': 3020, 'item5': 2055, 'item6': 3364, 'kills': 3, 'deaths': 6, 'assists': 3, 'largestKillingSpree': 2, 'largestMultiKill': 1, 'killingSprees': 1, 'longestTimeSpentLiving': 564, 'doubleKills': 0, 'tripleKills': 0, 'quadraKills': 0, 'pentaKills': 0, 'unrealKills': 0, 'totalDamageDealt': 71369, 'magicDamageDealt': 56332, 'physicalDamageDealt': 13037, 'trueDamageDealt': 1999, 'largestCriticalStrike': 0, 'totalDamageDealtToChampions': 10844, 'magicDamageDealtToChampions': 8926, 'physicalDamageDealtToChampions': 1918, 'trueDamageDealtToChampions': 0, 'totalHeal': 4006, 'totalUnitsHealed': 1, 'damageSelfMitigated': 6742, 'damageDealtToObjectives': 1055, 'damageDealtToTurrets': 1008, 'visionScore': 14, 'timeCCingOthers': 12, 'totalDamageTaken': 17255, 'magicalDamageTaken': 5282, 'physicalDamageTaken': 

Create function which grabs the championId and if they won or not. Use it on our set

In [5]:
# parseGame grabs the champion used for each user, and if that champ won or lost
def parseGame(data):
    out = []
    for game in data:
        # game is string, change type
        fixed = ast.literal_eval(game)
        # True = winner champ ids
        # False = loser champ ids
        stats = {
            True: [],
            False: []
        }
        for user in fixed:
            stats[user['stats']['win']].append(user['championId'])
        out.append(stats)
    return out
        
# Use parsegame on data
games = np.array(parseGame(data))
print(games[0:5])

[{True: [235, 81, 127, 84, 421], False: [7, 350, 266, 517, 110]}
 {True: [236, 245, 350, 110, 57], False: [235, 126, 517, 58, 15]}
 {True: [12, 429, 7, 64, 82], False: [235, 60, 223, 112, 58]}
 {True: [875, 41, 21, 154, 157], False: [111, 142, 236, 78, 98]}
 {True: [145, 875, 236, 34, 60], False: [62, 2, 517, 142, 350]}]


Now we build vectors using these numbers. The champ id is terribly coded and we need to hardcode it to 0-100 or such.

In [6]:
def getChamp(n):
    getChamp = {
        1: 0,
        2: 1,
        3: 2,
        4: 3,
        5: 4,
        6: 5,
        7: 6,
        8: 7,
        9: 8,
        10: 9,
        11: 10,
        12: 11,
        13: 12,
        14: 13,
        15: 14,
        16: 15,
        17: 16,
        18: 17,
        19: 18,
        20: 19,
        21: 20,
        22: 21,
        23: 22,
        24: 23,
        25: 24,
        26: 25,
        27: 26,
        28: 27,
        29: 28,
        30: 29,
        31: 30,
        32: 31,
        33: 32,
        34: 33,
        35: 34,
        36: 35,
        37: 36,
        38: 37,
        39: 38,
        40: 39,
        41: 40,
        42: 41,
        43: 42,
        44: 43,
        45: 44,
        48: 45,
        50: 46,
        51: 47,
        53: 48,
        54: 49,
        55: 50,
        56: 51,
        57: 52,
        58: 53,
        59: 54,
        60: 55,
        61: 56,
        62: 57,
        63: 58,
        64: 59,
        67: 60,
        68: 61,
        69: 62,
        72: 63,
        74: 64,
        75: 65,
        76: 66,
        77: 67,
        78: 68,
        79: 69,
        80: 70,
        81: 71,
        82: 72,
        83: 73,
        84: 74,
        85: 75,
        86: 76,
        89: 77,
        90: 78,
        91: 79,
        92: 80,
        96: 81,
        98: 82,
        99: 83,
        101: 84,
        102: 85,
        103: 86,
        104: 87,
        105: 88,
        106: 89,
        107: 90,
        110: 91,
        111: 92,
        112: 93,
        113: 94,
        114: 95,
        115: 96,
        117: 97,
        119: 98,
        120: 99,
        121: 100,
        122: 101,
        126: 102,
        127: 103,
        131: 104,
        133: 105,
        134: 106,
        136: 107,
        141: 108,
        142: 109,
        143: 110,
        145: 111,
        150: 112,
        154: 113,
        157: 114,
        161: 115,
        163: 116,
        164: 117,
        201: 118,
        202: 119,
        203: 120,
        222: 121,
        223: 122,
        235: 123,
        236: 124,
        238: 125,
        240: 126,
        245: 127,
        246: 128,
        254: 129,
        266: 130,
        267: 131,
        268: 132,
        350: 133,
        412: 134,
        420: 135,
        421: 136,
        427: 137,
        429: 138,
        432: 139,
        497: 140,
        498: 141,
        516: 142,
        517: 143,
        518: 144,
        523: 145,
        555: 146,
        875: 147,

    }
    return getChamp[n]

def getName(n):

    names = {
        0: 'Annie',
        1: 'Olaf',
        2: 'Galio',
        3: 'Twisted Fate',
        4: 'Xin Zhao',
        5: 'Urgot',
        6: 'LeBlanc',
        7: 'Vladimir',
        8: 'Fiddlesticks',
        9: 'Kayle',
        10: 'Master Yi',
        11: 'Alistar',
        12: 'Ryze',
        13: 'Sion',
        14: 'Sivir',
        15: 'Soraka',
        16: 'Teemo',
        17: 'Tristana',
        18: 'Warwick',
        19: 'Nunu & Willump',
        20: 'Miss Fortune',
        21: 'Ashe',
        22: 'Tryndamere',
        23: 'Jax',
        24: 'Morgana',
        25: 'Zilean',
        26: 'Singed',
        27: 'Evelynn',
        28: 'Twitch',
        29: 'Karthus',
        30: "Cho'Gath",
        31: 'Amumu',
        32: 'Rammus',
        33: 'Anivia',
        34: 'Shaco',
        35: 'Dr.Mundo',
        36: 'Sona',
        37: 'Kassadin',
        38: 'Irelia',
        39: 'Janna',
        40: 'Gangplank',
        41: 'Corki',
        42: 'Karma',
        43: 'Taric',
        44: 'Veigar',
        45: 'Trundle',
        46: 'Swain',
        47: 'Caitlyn',
        48: 'Blitzcrank',
        49: 'Malphite',
        50: 'Katarina',
        51: 'Nocturne',
        52: 'Maokai',
        53: 'Renekton',
        54: 'JarvanIV',
        55: 'Elise',
        56: 'Orianna',
        57: 'Wukong',
        58: 'Brand',
        59: 'LeeSin',
        60: 'Vayne',
        61: 'Rumble',
        62: 'Cassiopeia',
        63: 'Skarner',
        64: 'Heimerdinger',
        65: 'Nasus',
        66: 'Nidalee',
        67: 'Udyr',
        68: 'Poppy',
        69: 'Gragas',
        70: 'Pantheon',
        71: 'Ezreal',
        72: 'Mordekaiser',
        73: 'Yorick',
        74: 'Akali',
        75: 'Kennen',
        76: 'Garen',
        77: 'Leona',
        78: 'Malzahar',
        79: 'Talon',
        80: 'Riven',
        81: "Kog'Maw",
        82: 'Shen',
        83: 'Lux',
        84: 'Xerath',
        85: 'Shyvana',
        86: 'Ahri',
        87: 'Graves',
        88: 'Fizz',
        89: 'Volibear',
        90: 'Rengar',
        91: 'Varus',
        92: 'Nautilus',
        93: 'Viktor',
        94: 'Sejuani',
        95: 'Fiora',
        96: 'Ziggs',
        97: 'Lulu',
        98: 'Draven',
        99: 'Hecarim',
        100: "Kha'Zix",
        101: 'Darius',
        102: 'Jayce',
        103: 'Lissandra',
        104: 'Diana',
        105: 'Quinn',
        106: 'Syndra',
        107: 'AurelionSol',
        108: 'Kayn',
        109: 'Zoe',
        110: 'Zyra',
        111: "Kai'sa",
        112: 'Gnar',
        113: 'Zac',
        114: 'Yasuo',
        115: "Vel'Koz",
        116: 'Taliyah',
        117: 'Camille',
        118: 'Braum',
        119: 'Jhin',
        120: 'Kindred',
        121: 'Jinx',
        122: 'TahmKench',
        123: 'Senna',
        124: 'Lucian',
        125: 'Zed',
        126: 'Kled',
        127: 'Ekko',
        128: 'Qiyana',
        129: 'Vi',
        130: 'Aatrox',
        131: 'Nami',
        132: 'Azir',
        133: 'Yuumi',
        134: 'Thresh',
        135: 'Illaoi',
        136: "Rek'Sai",
        137: 'Ivern',
        138: 'Kalista',
        139: 'Bard',
        140: 'Rakan',
        141: 'Xayah',
        142: 'Ornn',
        143: 'Sylas',
        144: 'Neeko',
        145: 'Aphelios',
        146: 'Pyke',
        147: "Sett",

    }
    return names[n]

def getNum(n):

    names = {
        0: 'Annie',
        1: 'Olaf',
        2: 'Galio',
        3: 'Twisted Fate',
        4: 'Xin Zhao',
        5: 'Urgot',
        6: 'LeBlanc',
        7: 'Vladimir',
        8: 'Fiddlesticks',
        9: 'Kayle',
        10: 'Master Yi',
        11: 'Alistar',
        12: 'Ryze',
        13: 'Sion',
        14: 'Sivir',
        15: 'Soraka',
        16: 'Teemo',
        17: 'Tristana',
        18: 'Warwick',
        19: 'Nunu & Willump',
        20: 'Miss Fortune',
        21: 'Ashe',
        22: 'Tryndamere',
        23: 'Jax',
        24: 'Morgana',
        25: 'Zilean',
        26: 'Singed',
        27: 'Evelynn',
        28: 'Twitch',
        29: 'Karthus',
        30: "Cho'Gath",
        31: 'Amumu',
        32: 'Rammus',
        33: 'Anivia',
        34: 'Shaco',
        35: 'Dr.Mundo',
        36: 'Sona',
        37: 'Kassadin',
        38: 'Irelia',
        39: 'Janna',
        40: 'Gangplank',
        41: 'Corki',
        42: 'Karma',
        43: 'Taric',
        44: 'Veigar',
        45: 'Trundle',
        46: 'Swain',
        47: 'Caitlyn',
        48: 'Blitzcrank',
        49: 'Malphite',
        50: 'Katarina',
        51: 'Nocturne',
        52: 'Maokai',
        53: 'Renekton',
        54: 'JarvanIV',
        55: 'Elise',
        56: 'Orianna',
        57: 'Wukong',
        58: 'Brand',
        59: 'LeeSin',
        60: 'Vayne',
        61: 'Rumble',
        62: 'Cassiopeia',
        63: 'Skarner',
        64: 'Heimerdinger',
        65: 'Nasus',
        66: 'Nidalee',
        67: 'Udyr',
        68: 'Poppy',
        69: 'Gragas',
        70: 'Pantheon',
        71: 'Ezreal',
        72: 'Mordekaiser',
        73: 'Yorick',
        74: 'Akali',
        75: 'Kennen',
        76: 'Garen',
        77: 'Leona',
        78: 'Malzahar',
        79: 'Talon',
        80: 'Riven',
        81: "Kog'Maw",
        82: 'Shen',
        83: 'Lux',
        84: 'Xerath',
        85: 'Shyvana',
        86: 'Ahri',
        87: 'Graves',
        88: 'Fizz',
        89: 'Volibear',
        90: 'Rengar',
        91: 'Varus',
        92: 'Nautilus',
        93: 'Viktor',
        94: 'Sejuani',
        95: 'Fiora',
        96: 'Ziggs',
        97: 'Lulu',
        98: 'Draven',
        99: 'Hecarim',
        100: "Kha'Zix",
        101: 'Darius',
        102: 'Jayce',
        103: 'Lissandra',
        104: 'Diana',
        105: 'Quinn',
        106: 'Syndra',
        107: 'AurelionSol',
        108: 'Kayn',
        109: 'Zoe',
        110: 'Zyra',
        111: "Kai'sa",
        112: 'Gnar',
        113: 'Zac',
        114: 'Yasuo',
        115: "Vel'Koz",
        116: 'Taliyah',
        117: 'Camille',
        118: 'Braum',
        119: 'Jhin',
        120: 'Kindred',
        121: 'Jinx',
        122: 'TahmKench',
        123: 'Senna',
        124: 'Lucian',
        125: 'Zed',
        126: 'Kled',
        127: 'Ekko',
        128: 'Qiyana',
        129: 'Vi',
        130: 'Aatrox',
        131: 'Nami',
        132: 'Azir',
        133: 'Yuumi',
        134: 'Thresh',
        135: 'Illaoi',
        136: "Rek'Sai",
        137: 'Ivern',
        138: 'Kalista',
        139: 'Bard',
        140: 'Rakan',
        141: 'Xayah',
        142: 'Ornn',
        143: 'Sylas',
        144: 'Neeko',
        145: 'Aphelios',
        146: 'Pyke',
        147: "Sett",

    }
    rev = {y: x for x, y in names.items()}
    return rev[n]

x = np.zeros((games.size*2,148))
y = np.zeros((games.size*2,))
n = 0

# adjmat is an adjacency matrix of teammates
adjmat = np.zeros((148,148))
# enemat is graph of common enemies of champ
enemat = np.zeros((148,148))
# freq is the frequency of a champ getting picked
freq = np.zeros((148,1))

# for every game, check who won and who lost, add to data
for game in games:
    for team in game:
        
        # record game in data
        for champ in game[team]:
            # add relations to adjmat
            for tmate in game[team]:
                if tmate != champ:
                    adjmat[getChamp(champ)][getChamp(tmate)] += 1
                
            # add enemies' freqs
            for enemy in game[not team]:
                enemat[getChamp(champ)][getChamp(enemy)] += 1
            
            # add to freq
            freq[getChamp(champ)] += 1
                
            # add game as won and lost
            for i in range(2):
                y[n+(i*games.size)] = np.sign(i - 0.5)
                if team:
                    x[n+(i*games.size)][getChamp(champ)] = 1*np.sign(i - 0.5)
                else:
                    x[n+(i*games.size)][getChamp(champ)] = -1*np.sign(i - 0.5)
    n += 1      
print(x.shape[0])
print(x[0])
print(y[0])

185714
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.
  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.]
-1.0


Get samples

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [8]:
# We will need weight penalization because some combinations will have little to no games played
logR = LogisticRegression(penalty='l2',C=1.0,solver='liblinear',max_iter=1000)
logR.fit(x_train, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

Get test accuracy

In [9]:
score = logR.score(x_test, y_test)
print(score)

# now testing over, fit on all data
logR.fit(x, y)

0.5727977600689209


LogisticRegression(max_iter=1000, solver='liblinear')

A 57% accuracy is pretty good for our data. Now with the data we can make a function to suggest champions.

In [10]:
for row in adjmat:
    row /= np.sum(row)
    
for row in enemat:
    row /= np.sum(row)
    
freq /= np.sum(freq)
print(freq)

[[0.00072694]
 [0.01005443]
 [0.00457382]
 [0.00559908]
 [0.00239407]
 [0.00092941]
 [0.00980135]
 [0.0078736 ]
 [0.00307685]
 [0.0024673 ]
 [0.002631  ]
 [0.00420227]
 [0.00511014]
 [0.00100372]
 [0.00203436]
 [0.00506168]
 [0.00181251]
 [0.00201929]
 [0.00180282]
 [0.00258792]
 [0.02897864]
 [0.00637341]
 [0.00290993]
 [0.00995104]
 [0.00659418]
 [0.00460828]
 [0.00223791]
 [0.0017942 ]
 [0.00162943]
 [0.01014597]
 [0.00157127]
 [0.00089172]
 [0.0015982 ]
 [0.00176297]
 [0.00339132]
 [0.00087772]
 [0.00168543]
 [0.00427658]
 [0.01587105]
 [0.00539338]
 [0.00479459]
 [0.00053955]
 [0.00715312]
 [0.00600832]
 [0.00107049]
 [0.00171343]
 [0.00208067]
 [0.00790483]
 [0.01233218]
 [0.00260192]
 [0.00627002]
 [0.00533522]
 [0.00598786]
 [0.01441286]
 [0.00817838]
 [0.01898129]
 [0.00351517]
 [0.00463305]
 [0.00089064]
 [0.03665516]
 [0.00625925]
 [0.00931241]
 [0.00674388]
 [0.00047063]
 [0.00261699]
 [0.00124711]
 [0.01252173]
 [0.00044586]
 [0.00597278]
 [0.00869532]
 [0.01333052]
 [0.03

Adjmat is a net of all games played with i character as a teammate. Each row is a character, each column is that characters connections.

Enemat is a net of all games played as character i against characters in row j.

Freq is the frequency of a champ getting picked

Each of these matricies are averaged in order to get a percentage.

In [118]:
def getBestChamp(logR,myteam,enemy):
    
    # Fills in the already selected champions
    x = np.zeros((1, 148))
    for champ in myteam:
        x[0][getNum(champ)] = 1
    for champ in enemy:
        x[0][getNum(champ)] = -1
        
    # Finds champ with the percent which boosts your team winning the most
    score = 0
    out = 0
    for i in range(148):
        temp = 0        
        # score is a percent based off of
        # - Character pick rate
        # - Enemy selections
        # - Teammate selections
        # - Percent win after selection
        # The less selections of teammates and enemies, the more skewed the numbers are towards
        # character pick rate
        if x[0][i] == 0:
            x[0][i] = 1
            
            if (len(myteam) + len(enemy)) == 0:
                temp += 5*freq[i]
            
            for champ in myteam:
                temp += 120*adjmat[i][getNum(champ)]
            
            for champ in enemy:
                temp += 70*enemat[i][getNum(champ)]

            temp += 5*logR.predict_proba(x)[0][0]
            x[0][i] = 0
            
        
        if temp > score:
            score = temp
            out = i
    
    return getName(out)

# test picking teams, snake order
myteam = []
enemy = []
for i in range(6):    
    if (i == 0):
        # First pick is one pick
        newchamp = getBestChamp(logR,enemy,myteam)
        enemy.append(newchamp)
        print(enemy)
    elif (i == 5):
        # Last pick is one pick
        newchamp = getBestChamp(logR,myteam,enemy)
        myteam.append(newchamp)
        print(myteam)
    elif (i % 2 == 0):
        # Enemy gets to pick twice
        newchamp = getBestChamp(logR,enemy,myteam)
        enemy.append(newchamp)
        newchamp = getBestChamp(logR,enemy,myteam)
        enemy.append(newchamp)
        print(enemy)
    else:
        # My team gets to pick twice
        newchamp = getBestChamp(logR,myteam,enemy)
        myteam.append(newchamp)
        newchamp = getBestChamp(logR,myteam,enemy)
        myteam.append(newchamp)
        print(myteam)


['Ezreal']
["Kai'sa", 'Nautilus']
['Ezreal', 'Yuumi', 'Nasus']
["Kai'sa", 'Nautilus', 'Volibear', 'Yorick']
['Ezreal', 'Yuumi', 'Nasus', 'Karma', 'Skarner']
["Kai'sa", 'Nautilus', 'Volibear', 'Yorick', 'Riven']


System is just picking esoteric picks with high winrate all-around. 

In [119]:
print(getBestChamp(logR,["Ornn","Nocturne","Galio","Miss Fortune"],["Aatrox","LeeSin","LeBlanc","Aphelios","Taric"]))

Leona
